In [7]:
from javascript import require, On, Once, AsyncTask, once, off, globalThis
import json
import logging
import time

import llm
import building

import vision
import memory

mem = memory.Module()

MINECRAFT_VERSION = '1.20.2'

craft = require('./control_primitives/craftItem.js')
explore = require('./control_primitives/exploreUntil.js')
move = require('./control_primitives/goToPosition.js')
kill = require('./control_primitives/killMob.js')
collectPosition = require('./control_primitives/mineBlockPosition.js')
collectType = require('./control_primitives/mineBlockType.js')
pickupDroppedItem = require('./control_primitives/pickupDroppedItem.js')
place = require('./control_primitives/placeItem.js')
shoot = require('./control_primitives/shoot.js')
smelt = require('./control_primitives/smeltItem.js')
chest = require('./control_primitives/useChest.js')

Vec3 = require('vec3').Vec3
mineflayer = require('mineflayer')
pathfinder = require('mineflayer-pathfinder')
pvp = require('mineflayer-pvp').plugin
mcData = require('minecraft-data')(MINECRAFT_VERSION)
armorManager = require("mineflayer-armor-manager")
autoeat = require('mineflayer-auto-eat').plugin
collectblock = require('mineflayer-collectblock').plugin
hawkeye = require('minecrafthawkeye')
toolPlugin = require('mineflayer-tool').plugin
# mineflayerViewer = require('prismarine-viewer')

RANGE_GOAL = 1

# Load bot
def load_bot(username=None):
  # Create bot
  if not username:
    random_number = 176
    username = f'poo_bucket{random_number}'

  bot = mineflayer.createBot({ 
      'host': 'localhost',
      'port': 22222,
      'username': username, 
      'hideErrors': False 
  })

  once(bot, 'login')
  bot.chat('I spawned')
  #   mineflayerViewer(bot, { 'port': 3000, 'firstPerson': True })

  # Load plugins
  bot.loadPlugin(collectblock)
  bot.loadPlugin(hawkeye)
  bot.loadPlugin(pathfinder.pathfinder)
  bot.loadPlugin(pvp)
  bot.loadPlugin(toolPlugin)

  globalThis.mcData = mcData
  globalThis.pathfinder = pathfinder
  movements = pathfinder.Movements(bot, mcData)
  bot.pathfinder.setMovements(movements)

  # Auto armor
  bot.loadPlugin(armorManager)
  bot.armorManager.equipAll()

  # Auto eat
  bot.loadPlugin(autoeat)
  bot.autoEat.options.priority = 'foodPoints'
  bot.autoEat.options.startAt = 14
  bot.autoEat.options.bannedFood.push('golden_apple')
  
  return bot, movements

In [ ]:
bot, movements = load_bot('poo_bucket176')

In [ ]:
CHESTS = [mcData.blocksByName[name]['id'] for name in mcData.blocksByName if 'chest' in name]
BEDS = [mcData.blocksByName[name]['id'] for name in mcData.blocksByName if 'bed' in name]
CHAT_MESSAGES = []

def getEnvironmentInfo(bot):
    def distanceTo(pos1, pos2):
        return ((pos1.x - pos2.x) ** 2 + (pos1.y - pos2.y) ** 2 + (pos1.z - pos2.z) ** 2) ** 0.5

    def nearbyEntities(bot, max_distance={'mobs': 32, 'items': 16, 'players': 128}):
        nearby_mobs, nearby_droppeditems, nearby_players = [], [], []
        for entity in bot.entities:
            if bot.entities[entity]:
                # Mob
                if bot.entities[entity].type not in ['other', 'player', 'projectile'] and distanceTo(bot.entities[entity].position, bot.entity.position) < max_distance['mobs']:
                    if bot.canSeeBlock(bot.blockAt(bot.entities[entity].position)):
                        nearby_mobs.append(f'{bot.entities[entity].name} is {int(distanceTo(bot.entities[entity].position, bot.entity.position))} blocks away.')
                # Item
                elif bot.entities[entity].displayName == 'Item' and distanceTo(bot.entities[entity].position, bot.entity.position) < max_distance['items']:
                    if bot.canSeeBlock(bot.blockAt(bot.entities[entity].position)):

                        nearby_droppeditems.append(f'DroppedItem {mcData.items[bot.entities[entity].metadata[8].itemId].displayName} is {int(distanceTo(bot.entities[entity].position, bot.entity.position))} blocks away.')
                # Player
                elif bot.entities[entity].type == 'player' and bot.entities[entity].username != bot.username and distanceTo(bot.entities[entity].position, bot.entity.position) < max_distance['players']:
                    if bot.canSeeBlock(bot.blockAt(bot.entities[entity].position)):

                        nearby_players.append(f'Player: {bot.entities[entity].username} is {int(distanceTo(bot.entities[entity].position, bot.entity.position))} blocks away.')
        
        return nearby_mobs, nearby_droppeditems, nearby_players


    def nearbyBeds(bot, beds, max_distance=24, count=1):
        beds = bot.findBlocks({'matching': beds, 'maxDistance': max_distance, 'count': count})
        return [f'Bed at Vec3{bed.x, bed.y, bed.z} is {int(distanceTo(bed, bot.entity.position))} blocks away.' for bed in beds]

    def nearbyChests(bot, chests, max_distance=24, count=1):
        chests = bot.findBlocks({'matching': chests, 'maxDistance': max_distance, 'count': count})
        return [f'Chest at Vec3{chest.x, chest.y, chest.z} is {int(distanceTo(chest, bot.entity.position))} blocks away.' for chest in chests]
    
    nearby_mobs, nearby_droppeditems, nearby_players = nearbyEntities(bot, max_distance={'mobs': 64, 'items': 32, 'players': 128})

    nearby_beds = nearbyBeds(bot, BEDS, 24)
    nearby_chests = nearbyChests(bot, CHESTS, 24)

    return {
        'mobs': nearby_mobs,
        'droppedItems': nearby_droppeditems,
        'players': nearby_players,
        'beds': nearby_beds,
        'chests': nearby_chests
    }

def getWorldInfo(bot):
    # Time
    time = bot.time.timeOfDay
    if 0 <= time < 9000:
        time_description = 'in the Morning'
    elif 9000 <= time < 12000:
        time_description = 'at Dusk'
    else:
        time_description = 'at Night'
    
    # Weather
    weather = 'It is clear.' 
    if bot.rainState:
        weather = 'It is raining and thundering!' if bot.thunderState else 'It is raining!'
    
    return {
        'time': f'It is currently {time} {time_description}',
        'weather': weather,
        'biome': mcData.biomes[bot.blockAt(bot.entity.position).biome.id].displayName
    }

def getPlayerState(bot):
    # Inventory
    inventory = []
    for item in bot.inventory.items():
        inventory.append(f'{item.displayName} x{item.count}')
    if len(inventory) == 0: 
        inventory = 'Empty'

    # Held Item
    heldItem = 'Nothing'
    if bot.heldItem:
        heldItem = bot.heldItem.displayName

    return {
        'position': f'Vec3{bot.entity.position.x, bot.entity.position.y, bot.entity.position.z}',
        'pitch': bot.entity.pitch,
        'yaw': bot.entity.yaw,
        'health': bot.health,
        'hunger': bot.food,
        'heldItem': heldItem,
        'inventory': inventory
    }

@On(bot, 'chat')
def handleMsg(this, sender, message, *args):
    if sender and (sender != bot.username):
        if 'come' in message:
            player = bot.players[sender]
            target = player.entity
            if not target:
                bot.chat("I don't see you !")
                return
            pos = target.position
            bot.pathfinder.setMovements(movements)
            bot.pathfinder.setGoal(pathfinder.goals.GoalNear(pos.x, pos.y, pos.z, RANGE_GOAL))
        if 'stop' in message:
            off(bot, 'chat', handleMsg)
    else:
        if sender == bot.username:
            sender = 'Minecraft Bot (You)'
        CHAT_MESSAGES.append(f'{sender} said {message}')

def getPromptInfo(bot):
    world_info = getWorldInfo(bot)
    environment_info = getEnvironmentInfo(bot)
    player_state = getPlayerState(bot)

    # with open('./responses/responses.txt', 'r') as file:
    #     visual_information = file.read()

    return {
        'world_info': world_info,
        'environment_info': environment_info,
        'player_state': player_state,
        'recent_chat_messages': CHAT_MESSAGES[::-1][:5]
    }

In [8]:
with open('descriptions.json', 'r') as file:
    bot_functions = json.load(file)
    
llms = llm.get_llms()

logging.basicConfig(filename='./process_log.txt', level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

findAndParseJsonLikeText = require('json-like-parse')

In [ ]:
import perception_llms
llms = perception_llms.get_llms()
task = "What should my first task be?"
perception_functions = json.loads(open('perception_functions.json').read())
print(f'{task=}')

In [ ]:
relevant_reasoning_modules = llms['select'].invoke({
    'task': task
}).content
print(f'{relevant_reasoning_modules=}')
adapted_reasoning_modules = llms['adapt'].invoke({
    'task': task,
    'relevant_reasoning_modules': relevant_reasoning_modules
}).content
print(f'{adapted_reasoning_modules=}')
implemented_reasoning_modules = llms['implement'].invoke({
    'task': task,
    'adapted_reasoning_modules': adapted_reasoning_modules,
    'perception_functions': perception_functions
}).content
subtasks = findAndParseJsonLikeText(implemented_reasoning_modules)[0]
print(f'{subtasks=}')

In [ ]:
def findBlockType(name, maxDistance=32):
    block = bot.findBlock({
        'matching': mcData.blocksByName[name].id,
        'maxDistance': maxDistance
    })
    
    return block

In [ ]:
environment = {'environment': str(getPromptInfo(bot)), 'Additional Information': []}
print(environment)
attempt = 0
previous_attempts = []
while attempt <= 3:
    response = llms['perceive'].invoke({
        'task': task,
        'subtasks': subtasks,
        'environment': environment,
        'current_attempt': attempt,
        'perception_functions': perception_functions,
        'previous_attempts': previous_attempts
    }).content
    print(f'{response=}')

    if 'subtask answer' in response.lower():
        subtasks = response.split('SUBTASK ANSWER')[1]
        print(f'{subtasks=}')

    if 'completed' in response.lower():
        print(f'Completed task: {response.split("COMPLETED")[1]}')
        break

    # revisit this portion
    else:
        try:
            global output
            output = None

            exec(f"global output\noutput = {response}")
            if 'findBlockType' in response:
                if output:
                    response = f'{response} was found in the world at: {output.position}!'
                else:
                    response = f'{response} was not found in this location!'
            
            else:
                environment['Additional Information'].append(output)
                response += f': {output}'

        except:
            environment['environment'] = str(getPromptInfo(bot))

    previous_attempts.append(response)
    print(previous_attempts)

    attempt += 1

In [ ]:
print(response)

In [2]:
mem.query_memories('Who am I?')

Number of requested results 10 is greater than number of elements in index 9, updating n_results = 9


'Based on the information collected, you are Gary :D.'